In [1]:
# import deps
import pandas as pd
import numpy as np
import re

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки data. Для таблицы transactions используйте только первые n=1000000 строк. Обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

In [ ]:
!pip install PyDrive

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# загружаем файлики датасетов и импортируем сами датасеты
downloaded = drive.CreateFile({'id': '1ugyqERH-9MUs6R9hiJmhtiI3unBsNh0n'})
downloaded.GetContentFile('transactions.csv')

transactions_ds = pd.read_csv('transactions.csv', sep=',', nrows=10**6)

downloaded = drive.CreateFile({'id': '1-TW6ae5_0iVb0g17RhWlmTO4cg6FAEAe'})
downloaded.GetContentFile('gender_train.csv')

gender_train_ds = pd.read_csv('gender_train.csv', sep=',')

downloaded = drive.CreateFile({'id': '1gWt_IdY2P8BcFUM_ww6wy5zmmL7IV3su'})
downloaded.GetContentFile('tr_mcc_codes.csv')

tr_mcc_codes_ds = pd.read_csv('tr_mcc_codes.csv', sep=';')

downloaded = drive.CreateFile({'id': '1N1qzjRtJQx5nBxSaheAk7dYDKOiOPJ0K'})
downloaded.GetContentFile('tr_types.csv')

tr_types_ds = pd.read_csv('tr_types.csv', sep=';')

Выведем head всех датасетов, чтобы убедиться что всё хорошо:


In [5]:
transactions_ds.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN
1,39026145,1 10:19:29,6011,7010,56147.89,NaN
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN
4,39026145,2 15:33:42,5499,1010,-920.83,NaN


In [6]:
gender_train_ds.head()

,customer_id,gender
0,10928546,1
1,69348468,1
2,61009479,0
3,74045822,0
4,27979606,1


In [7]:
tr_mcc_codes_ds.head()

,mcc_code,mcc_description
0,742,Ветеринарные услуги
1,1711,"Генеральные подрядчики по вентиляции, теплосна..."
2,1731,Подрядчики по электричеству
3,1799,"Подрядчики, специализированная торговля — нигд..."
4,2741,Разнообразные издательства/печатное дело


In [8]:
tr_types_ds.head()

,tr_type,tr_description
0,3200,Плата за предоставление услуг посредством моби...
1,3210,Плата за предоставление отчета по счету карты ...
2,3800,Плата за обслуживание банковской карты (за пер...
3,4000,Плата за получение наличных в Сбербанке
4,4001,Плата за получение наличных в Сбербанке (в дру...


## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'

In [9]:
sub_tr_type_ds = tr_types_ds.sample(n=1000, replace=True)

sub_tr_type_ds.head()

,tr_type,tr_description
102,1200,Покупка. Зарубеж. ТУ
150,2990,Списание по требованию
88,2440,н/д
109,2021,Выдача наличных через POS Сбербанка (в других ТБ)
38,7000,Пополнение счета в подразд. СБ РФ


In [10]:
regex = re.compile("POS.*|АТМ.*")

contains = sub_tr_type_ds.tr_description.str.contains(regex)

contains.sum() / sub_tr_type_ds.tr_description.size

0.418

## Задание 2 


1. Для столюца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости.

In [11]:
# считаем частоту для всех
freq = transactions_ds.tr_type.value_counts(normalize=True)

freq

1010    0.231117
2010    0.151166
7070    0.149006
1110    0.137658
1030    0.118975
          ...   
4061    0.000004
4096    0.000004
8146    0.000003
8100    0.000002
1510    0.000002
Name: tr_type, Length: 72, dtype: float64

In [12]:
# выводим первые 10
freq[:10]

1010    0.231117
2010    0.151166
7070    0.149006
1110    0.137658
1030    0.118975
2370    0.049830
7010    0.028972
7030    0.026078
7071    0.015781
1100    0.015476
Name: tr_type, dtype: float64

## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите разницу для этих клиентов между суммой расходов и суммой приходов

In [13]:
# получаем частоту встречаемости каждого клиента
freq_customer = transactions_ds.customer_id.value_counts(normalize=True)

freq_customer

70780820    0.088781
63233186    0.018293
44552122    0.012325
32489134    0.010146
54186907    0.009869
              ...   
32242229    0.000004
94328553    0.000004
75010111    0.000003
42964024    0.000002
21283235    0.000002
Name: customer_id, Length: 1892, dtype: float64

In [18]:
max_neg_customers_id = transactions_ds[transactions_ds.amount < 0].groupby("customer_id").sum().amount.idxmin()
max_pos_customers_id = transactions_ds[transactions_ds.amount > 0].groupby("customer_id").sum().amount.idxmax()

print("id neg:", max_neg_customers_id)
print("id pos:", max_pos_customers_id)

max_neg_customers = transactions_ds[transactions_ds.amount < 0].groupby("customer_id").sum().amount.min()
max_pos_customers = transactions_ds[transactions_ds.amount > 0].groupby("customer_id").sum().amount.max()

print("sum neg:", max_neg_customers)
print("sum pos:", max_pos_customers)

print("diff:", abs(max_neg_customers) - abs(max_pos_customers))

id neg: 70780820
id pos: 70780820
sum neg: -1249952204.789996
sum pos: 1248114886.810022
diff: 1837317.979973793


## Задание 4
1. Найдите среднее арифметическое и медиану по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по всем типам транзакций для клиентов из задания 3

In [22]:
freq_avg = freq[:10].mean()
freq_median = freq[:10].median()

print("avg: ", freq_avg)
print("median: ", freq_median)

avg:  0.09240590000000001
median:  0.08440249999999999


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [21]:
transactions_ds = pd.merge(transactions_ds, gender_train_ds, how='left')
transactions_ds = pd.merge(transactions_ds, tr_mcc_codes_ds, how='inner')
transactions_ds = pd.merge(transactions_ds, tr_types_ds, how='inner')
transactions_ds.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [23]:
transactions_ds.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ


In [34]:
avg_pos_gender_one = transactions_ds[(transactions_ds.amount > 0) & (transactions_ds.gender == 1.0)].amount.mean()
avg_pos_gender_two = transactions_ds[(transactions_ds.amount > 0) & (transactions_ds.gender == 0.0)].amount.mean()

print("pos diff: ", abs(abs(avg_pos_gender_one) - abs(avg_pos_gender_two)))

pos diff:  63366.57104801515


In [35]:
avg_neg_gender_one = transactions_ds[(transactions_ds.amount < 0) & (transactions_ds.gender == 1.0)].amount.mean()
avg_neg_gender_two = transactions_ds[(transactions_ds.amount < 0) & (transactions_ds.gender == 0.0)].amount.mean()

print("neg diff: ", abs(abs(avg_neg_gender_one) - abs(avg_neg_gender_two)))

neg diff:  32718.05492021877


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 транзакций для мужчин и для женщин, наименьших среди всех транзакций по полученным значениям "max_income". 
2. Выделите среди них те, которые встречаются одновременно и у мужчин, и у женщин

## Задание 7

1. Найдите суммы затрат по каждой категории для мужчин и для женщин. 
2. Найдите топ 5 категорий с самыми большими относительными модулями разности для в тратах для разных полов.

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте ночные поступления для мужчин и женщин. Ночные - это примерно 00-06 часов. 

In [53]:
# выделяем tr_hour
transactions_ds_cp = transactions_ds.copy()
transactions_ds_cp["tr_hour"] = transactions_ds_cp.apply(lambda item: item.tr_datetime.split(":")[0].split()[1], axis=1).astype(int)

transactions_ds_cp.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description,tr_hour
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,10
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,7
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,7
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,8
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,14


In [54]:
tr_hour = transactions_ds_cp.tr_hour
amount = transactions_ds_cp.amount
gender = transactions_ds_cp.gender

night_pos_values = transactions_ds_cp[(0 <= tr_hour) & (tr_hour <= 6) & (amount > 0)]

night_pos_values.groupby(gender).amount.sum()

gender
0.0    5.806437e+08
1.0    9.226917e+08
Name: amount, dtype: float64